## Overview

Codey models are text-to-code models from Google AI, trained on a massive code related dataset. You can generate code related responses for different scenarios such as writing functions, unit tests, debugging, autocompleting and etc. This notebook is to show you how to call REST APIs and Python SDKs of different codey models including code-bision, code-gecko, and codechat-bison.

The notebook is structured as follows:

1. We will explain what code-bison can do and how to use REST APIs and Python SDKs of code-bison with examples.

2. We will explain what code-gecko can do and how to use REST APIs and Python SDKs of code-gecko with examples.

3. We will explain what codechat-bison can do and how to use REST APIs and Python SDKs of codechat-bison with examples.

Refer to the doc for the latest codey model overview: https://cloud.google.com/vertex-ai/docs/generative-ai/code/code-models-overview

Credit to jarekk@ for his work at 01-foundation-models-walkthrough notebook


* Contributor: leip@
* Date: 09/22/23

## Prep Work

In [ ]:
# @title Install Libraries
import sys

if 'google.colab' in sys.modules:
    ! pip install google-cloud-aiplatform
    ! pip install google-cloud-discoveryengine
    from google.colab import auth as google_auth
    google_auth.authenticate_user()

## Code Models

Vertex AI includes a suite of models that work with code. Together these code models are referred to as the Vertex AI Codey APIs. The Vertex AI Codey APIs include the following:

* The **code generation API** - Generates code based on a natural language description of the desired code. For example, it can generate a unit test for a function. The code generation API supports the code-bison model.

* The **code chat API** - Can power a chatbot that assists with code-related questions. For example, you can use it for help debugging code. The code chat API supports the codechat-bison model.

* The **code completion API** - Provides code autocompletion suggestions as you write code. The API uses the context of the code you're writing to make its suggestions. The code completion API supports the code-gecko model.

Refer to the doc for the latest codey model overview: https://cloud.google.com/vertex-ai/docs/generative-ai/code/code-models-overview


The Codey APIs support the following programming languages:
https://cloud.google.com/vertex-ai/docs/generative-ai/code/code-models-overview#supported_coding_languages

### **code-bison** model

The **code-bison** model supports generating code using a natural language description. Refer to code-bison doc for the latest updates:

Some common use cases for code generation are:

* Unit tests - Design a prompt to request a unit test for a function.
* Write a function - Pass a problem to the model to get a function that solves the problem.
* Create a class - Use a prompt to describe the purpose of a class and have code that defines the class returned.

#### Stable version

code-bison@001

#### Model properties
* Max input tokens: 6144
* Max ouptut tokens: 2048

#### REST API

##### HTTP request
`POST https://us-central1-aiplatform.googleapis.com/v1/projects/PROJECT_ID/locations/us-central1/publishers/google/models/code-bison:predict`

##### Request body
```
{
  "instances": [
    { "prefix": string }
  ],
  "parameters": {
    "temperature": number,
    "maxOutputTokens": integer,
    "candidateCount": integer,
    "stopSequences": [ string ]
  }
}
```

##### Request response

```
{
  "predictions": [
    {
      "content": string,
      "score": float,
      "citationMetadata": {
        "citations": [
          {
            "startIndex": integer,
            "endIndex": integer,
            "url": string,
            "title": string,
            "license": string,
            "publicationDate": string
          }
        ]
      },
      "safetyAttributes":{
        "categories": [],
        "blocked": false,
        "scores": []
      },
      "score": float
    }
  ]
}
```

Refer to the [API reference](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/code-generation) for the detailed information.


#### Using Python SDK

Create the model.

In [ ]:
import vertexai
from vertexai.language_models import CodeGenerationModel
VERTEX_API_PROJECT = 'certain-haiku-391918'
VERTEX_API_LOCATION = 'us-central1'
vertexai.init(project=VERTEX_API_PROJECT, location=VERTEX_API_LOCATION)
code_generation_model = CodeGenerationModel.from_pretrained("code-bison@001")

Invoke the model with parameters and prefix

- Write a function to check leap year with pseudo code

In [ ]:
prefix = "Write a pseudo code for a function that checks if a year is a leap year."

parameters = {
    "temperature": 0.2,
    "max_output_tokens": 512
}

response = code_generation_model.predict(
        prefix=prefix, **parameters
)

print(type(response))

<class 'vertexai.language_models.TextGenerationResponse'>


The response is a `TextGenerationResponse object`.

In [ ]:
print(response.text)

**Algorithm**

1. **Input** the year to be checked.
2. **If** the year is divisible by 4, then:
    * **If** the year is divisible by 100, then:
        * **If** the year is divisible by 400, then the year is a leap year.
        * **Else** the year is not a leap year.
    * **Else** the year is a leap year.
3. **Else** the year is not a leap year.

**Output** the result of the check.


- Write a function to check leap year with Python


In [ ]:
prefix = "Write a Python function that checks if a year is a leap year."

parameters = {
    "temperature": 0.2,
    "max_output_tokens": 512
}

response = code_generation_model.predict(
        prefix=prefix, **parameters
)

print(response)

```python
def is_leap_year(year):
  """
  Determines whether a year is a leap year.

  Args:
    year: The year to check.

  Returns:
    True if the year is a leap year, False otherwise.
  """

  # A year is a leap year if it is divisible by 4, unless it is divisible by 100
  # unless it is also divisible by 400.

  if year % 4 == 0:
    if year % 100 == 0:
      return year % 400 == 0
    else:
      return True
  else:
    return False
```


### **code-gecko** model

The **code-gecko** model supports code completion. It completes code that was typed by a user.

Some common use cases for code generation are:

* Write code faster: Use the code-geckomodel to write code faster by taking advantage of code suggested for you.

* Minimize bugs in code: Use code suggestions that you know are syntactically correct to avoid errors. Code completion helps you minimize the risk of accidentally introducing bugs that can occur when you write code quickly.

#### Stable version

code-gecko@001

#### Model properties
* Max input tokens: 2048
* Max ouptut tokens: 64

#### REST API

##### HTTP request
`POST https://us-central1-aiplatform.googleapis.com/v1/projects/PROJECT_ID/locations/us-central1/publishers/google/models/code-gecko:predict`

##### Request body
```
{
  "instances": [
    { "prefix": string,
      "suffix": string }
  ],
  "parameters": {
    "temperature": number,
    "maxOutputTokens": integer,
    "candidateCount": integer,
    "stopSequences": [ string ]
  }
}
```

##### Request response

```
{
  "predictions": [
    {
      "content": string,
      "citationMetadata": {
        "citations": [
          {
            "startIndex": integer,
            "endIndex": integer,
            "url": string,
            "title": string,
            "license": string,
            "publicationDate": string
          }
        ]
      },
      "safetyAttributes": {
        "categories": [ string ],
        "blocked": boolean,
        "scores": [ float ]
      },
      "score": float
    }
  ]
}
```

Refer to the [API reference](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/code-completion) for the detailed information.

#### Using Python SDK

Create the model.

In [ ]:
from vertexai.language_models import CodeGenerationModel

code_completion_model = CodeGenerationModel.from_pretrained("code-gecko@001")

Invoke the model with parameters and prefix

- Complete code of reverse string function

In [ ]:
prefix = "def reverse_string(s):"

parameters = {
        "temperature": 0.2,
        "max_output_tokens": 64,
}

response = code_completion_model.predict(prefix=prefix, **parameters)

print(type(response))

<class 'vertexai.language_models.TextGenerationResponse'>


The response is a TextGenerationResponse object.

In [ ]:
print(response.text)
print('score: ', response._prediction_response.predictions[0]['score'])


    return s[::-1]


def reverse_string_2(s):
score:  -2.4987316131591797


### **codechat-bison** model

The **codechat-bison** model supports multi-turn conversations that are specialized for code. The model allows developers to chat with a chatbot for help with code-related questions.

Some common use cases for code generation are:

* Get help about code: Get help with questions you have about code, such as questions about an API, syntax in a supported programming language, or which version of a library is required for code you're writing.

* Debugging: Get help with debugging code that doesn't compile or that contains a bug.

* Documentation: Get help understanding code so you can document it accurately.

* Learn about code: Get help learning about code you're not familiar with.

#### Stable version

codechat-bison@001

#### Model properties
* Max input tokens: 6144
* Max ouptut tokens: 2048

#### REST API

##### HTTP request
`POST https://us-central1-aiplatform.googleapis.com/v1/projects/PROJECT_ID/locations/us-central1/publishers/google/models/codechat-bison:predict`

##### Request body
```
{
  "instances": [
    {
      "context": string,
      "messages": [
        {
          "content": string,
          "author": string
        }
  ],
  "parameters": {
    "temperature": number,
    "maxOutputTokens": integer,
    "candidateCount": integer
  }
}


```

##### Request response

```
{
  "predictions": [
    {
      "candidates": [
        {
          "author": string,
          "content": string
        }
      ],
      "citationMetadata": {
        "citations": [
          {
            "startIndex": integer,
            "endIndex": integer,
            "url": string,
            "title": string,
            "license": string,
            "publicationDate": string
          }
        ]
      },
      "safetyAttributes": {
        "categories": [],
        "blocked": false,
        "scores": []
      },
      "score": float
    }
  ]
}
```

Refer to the [API reference](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/code-chat) for the detailed information.

#### Using Python SDK

Create the model.

In [ ]:
from vertexai.language_models import CodeChatModel

code_chat_model = CodeChatModel.from_pretrained("codechat-bison@001")

Create a chat session.

In [ ]:
chat = code_chat_model.start_chat()

Invoke the model with parameters and prefix

- Modify functions with codechat-bison

In [ ]:
message = "Please help write a Python function to calculate the min of two numbers"

parameters = {
    "temperature": 0.2,
    "max_output_tokens": 1024
}

response = chat.send_message(message,**parameters)

print(response)

The following Python function calculates the minimum of two numbers:

```
def min(a, b):
  """
  Calculates the minimum of two numbers.

  Args:
    a: The first number.
    b: The second number.

  Returns:
    The smaller of the two numbers.
  """

  if a < b:
    return a
  else:
    return b
```

This function takes two numbers as input and returns the smaller of the two numbers. The function first compares the two numbers and returns the smaller number. If the two numbers are equal, the function returns either of the two numbers.


Continue a conversation.

In [ ]:
message = "Modify the function to multiple a result by 2."

response = chat.send_message(message)

print(response)

Sure, here is the modified function:

```
def min(a, b):
  """
  Calculates the minimum of two numbers and multiplies the result by 2.

  Args:
    a: The first number.
    b: The second number.

  Returns:
    The smaller of the two numbers multiplied by 2.
  """

  if a < b:
    return a * 2
  else:
    return b * 2
```

This function takes two numbers as input and returns the smaller of the two numbers multiplied by 2. The function first compares the two numbers and returns the smaller number. If the two numbers are equal, the function returns either of the two numbers multiplied by 2.
